In [9]:
import dash
from dash.dependencies import Output, Input
from dash import dcc, html, dcc
from datetime import datetime
import json
import plotly.graph_objs as go
from collections import deque
from flask import Flask, request

Run this Python script and visit http://localhost:8000/ on your computer. Then you have to enter the correct Push URL in Sensor Logger on your phone under the settings page. To find out the localhost of the device you are running the websever on, you can, for example, do something like this in Python.

In [10]:
import socket
hostname = socket.gethostname()
print(socket.gethostbyname(hostname))

10.30.35.253


For example, if it returns 192.168.1.168, then you want to enter http://192.168.1.168:8000/data in Sensor Logger. Use the "Tap to Test Pushing" button to test whether Sensor Logger can properly reach the endpoint. If you get a 200 response, then you are good to go! Start a recording as usual, and you should begin to see data being streamed in:

In [ ]:
server = Flask(__name__)
app = dash.Dash(__name__, server=server)

MAX_DATA_POINTS = 1000
UPDATE_FREQ_MS = 100

time = deque(maxlen=MAX_DATA_POINTS)
accel_x = deque(maxlen=MAX_DATA_POINTS)
accel_y = deque(maxlen=MAX_DATA_POINTS)
accel_z = deque(maxlen=MAX_DATA_POINTS)

app.layout = html.Div(
	[
		dcc.Markdown(
			children="""
			# Live Sensor Readings
			Streamed from Sensor Logger: tszheichoi.com/sensorlogger
		"""
		),
		dcc.Graph(id="live_graph"),
		dcc.Interval(id="counter", interval=UPDATE_FREQ_MS),
	]
)


@app.callback(Output("live_graph", "figure"), Input("counter", "n_intervals"))
def update_graph(_counter):
	data = [
		go.Scatter(x=list(time), y=list(d), name=name)
		for d, name in zip([accel_x, accel_y, accel_z], ["X", "Y", "Z"])
	]

	graph = {
		"data": data,
		"layout": go.Layout(
			{
				"xaxis": {"type": "date"},
				"yaxis": {"title": "Acceleration ms<sup>-2</sup>"},
			}
		),
	}
	if (
		len(time) > 0
	):  #  cannot adjust plot ranges until there is at least one data point
		graph["layout"]["xaxis"]["range"] = [min(time), max(time)]
		graph["layout"]["yaxis"]["range"] = [
			min(accel_x + accel_y + accel_z),
			max(accel_x + accel_y + accel_z),
		]

	return graph


@server.route("/data", methods=["POST"])
def data():  # listens to the data streamed from the sensor logger
	if str(request.method) == "POST":
		print(f'received data: {request.data}')
		data = json.loads(request.data)
		for d in data['payload']:
			if (
				d.get("name", None) == "accelerometer"
			):  #  modify to access different sensors
				ts = datetime.fromtimestamp(d["time"] / 1000000000)
				if len(time) == 0 or ts > time[-1]:
					time.append(ts)
					# modify the following based on which sensor is accessed, log the raw json for guidance
					accel_x.append(d["values"]["x"])
					accel_y.append(d["values"]["y"])
					accel_z.append(d["values"]["z"])
	return "success"


if __name__ == "__main__":
	app.run_server(port=8000, host="0.0.0.0")

In [11]:
# logs sensorlogger input to log.js
# run as:
# flask run --host=0.0.0.0 --port=<portnumber>

# in sensorlogger, configure HTTP push as:
# http://<thisip>:<portnumber>/sensorlogger

from datetime import datetime
import json

from flask import Flask, request

app = Flask(__name__)

fd = open("log.js", "w")



@app.route("/sensorlogger", methods=["POST"])
def data():  # listens to the data streamed from the sensor logger
    if str(request.method) == "POST":
        print(f'received data: {request.data}')
        data = json.loads(request.data)
        fd.write(json.dumps(data, indent=4))
        fd.write("\n\n")
    return "success"

In [ ]:
export INFLUXDB_TOKEN=5SNGsMtpQEPCFMSduETRGq6strEi4a9FBZYhQ0EOI6Q4OKhcIDqdDkQ7dEqNdgo_AeyDISEd924MmoomFo9HiA==

In [9]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = os.environ.get("INFLUXDB_TOKEN")
org = "amer.altizini@gmail.com"
url = "https://europe-west1-1.gcp.cloud2.influxdata.com"

client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

bucket="Amer"

In [10]:
write_api = client.write_api(write_options=SYNCHRONOUS)

for value in range(5):
  point = (
    Point("measurement1")
    .tag("tagname1", "tagvalue1")
    .field("field1", value)
  )
  write_api.write(bucket=bucket, org="amer.altizini@gmail.com", record=point)
  time.sleep(1) # separate points by 1 second

ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 14 Dec 2022 09:45:59 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '55', 'Connection': 'keep-alive', 'trace-id': '67ca25d31f3ecc06', 'trace-sampled': 'false', 'x-platform-error-code': 'unauthorized', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Influxdb-Request-ID': 'ee4fe1a91c74dd78c1894e5cd951a6fa', 'X-Influxdb-Build': 'Cloud'})
HTTP response body: {"code":"unauthorized","message":"unauthorized access"}


In [18]:
bucket="Amer"

write_api = client.write_api(write_options=SYNCHRONOUS)

for value in range(5):
  point = (
    Point("measurement1")
    .tag("tagname1", "tagvalue1")
    .field("field1", value)
  )
  write_api.write(bucket=bucket, org="amer.altizini@gmail.com", record=point)
  time.sleep(1) # separate points by 1 second

ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 07 Dec 2022 15:50:32 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '55', 'Connection': 'keep-alive', 'trace-id': '705bc4d2d803dd40', 'trace-sampled': 'false', 'x-platform-error-code': 'unauthorized', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Influxdb-Request-ID': '5b3d7be590590baac9919011799df770', 'X-Influxdb-Build': 'Cloud'})
HTTP response body: {"code":"unauthorized","message":"unauthorized access"}


In [ ]:
# HTTP write listener for sensorlogger
# https://github.com/influxdata/telegraf/blob/release-1.23/plugins/inputs/http_listener_v2/README.md
[[inputs.http_listener_v2]]
http_header_tags = { "HTTP_HEADER" = "TAG_NAME" }
paths = ["/sensorlogger"]
service_address = ":4711"
# do not save path as http_listener_v2_path tag
path_tag = false
data_source = "body"
data_format = "json_v2"

# massage the JSON with https://github.com/influxdata/telegraf/tree/master/plugins/parsers/json_v2
[[inputs.http_listener_v2.json_v2]]
measurement_name = "sensorlogger"
timestamp_format = "unix_ns"
timestamp_path = "payload.#.time"

[[inputs.http_listener_v2.json_v2.tag]]
path = "sessionId"           # A string with valid GJSON path syntax to a non-array/non-object value
rename = "session" # A string with a new name for the tag key
## Setting optional to true will suppress errors if the configured Path doesn't match the JSON
optional = false

[[inputs.http_listener_v2.json_v2.tag]]
path = "deviceId"           # A string with valid GJSON path syntax to a non-array/non-object value
rename = "device" # A string with a new name for the tag key
## Setting optional to true will suppress errors if the configured Path doesn't match the JSON
optional = false

[[inputs.http_listener_v2.json_v2.object]]
disable_prepend_keys = true
excluded_keys = ["time"]
path = "payload"
tags = ["name"]

# InfluxDB V2 writer
# https://github.com/influxdata/telegraf/blob/release-1.23/plugins/outputs/influxdb_v2/README.md
[[outputs.influxdb_v2]]
bucket = "Amer"
organization = "mah"
token = "ExOgC_Cjgv4XOHnQrJqHeFlUqy7bVXmlSSbDc4bAPpDZm62U2b3aETl4IfIgSwug-VHlyc7BFaJWMbKo932l2w=="
urls = ["http://127.0.0.1:8086"]

# only for debugging
# show line protocol output in telegraf stdout

[[outputs.file]]
files = ["stdout"]